# Comparison of Male & Female

Words, words, words. 

In [1]:
%matplotlib inline

from scipy.stats import ttest_1samp, ttest_ind

from om.meg.single import MegSubj
from om.meg.group import MegGroup

from om.plts.meg import *

from om.core.db import OMDB
from om.core.osc import Osc

In [2]:
# Get database object, and list of subjects
db = OMDB()
sub_nums, source = db.check_dat_files(dat_type='fooof', dat_source='HCP', verbose=False)

In [3]:
# Drop outlier subject
sub_nums = list(set(sub_nums) - set([662551]))

### Male vs. Female maps

Note: for age and sample size balancing, loading drops a subset of both male & female subjects.

In [4]:
# Create oscillation index
osc = Osc()
osc.add_band('Theta', [3, 7])
osc.add_band('Alpha', [7, 14])
osc.add_band('Beta', [15, 30])

In [5]:
# Initialize MegGroup object
male_group = MegGroup(db, osc)
female_group = MegGroup(db, osc)

# Add subjects to meg_group, split up by Male / Female
for i, subj in enumerate(sub_nums):
    
    meg_subj = MegSubj(OMDB(), source[i], osc)         # Initialize MegSubj object
    meg_subj.import_fooof(subj, get_demo=True)         # Import subject data
    meg_subj.all_oscs(verbose=False)                   # Create vectors of all oscillations
    meg_subj.osc_bands_vertex()                        # Get oscillations per band per vertex
    meg_subj.peak_freq(dat='band', avg='median')       # Calculate peak frequencies
    
    if meg_subj.sex is 'M':
        
        if meg_subj.age < 25:
            continue
        
        male_group.add_subject(meg_subj, add_all_oscs=True, add_vertex_bands=True, add_peak_freqs = True,
                               add_vertex_oscs=False, add_vertex_exponents=True, add_demo=True)
        
    elif meg_subj.sex is 'F':
        
        if meg_subj.age > 34:
            continue
        
        female_group.add_subject(meg_subj, add_all_oscs=True, add_vertex_bands=True, add_peak_freqs = True,
                               add_vertex_oscs=False, add_vertex_exponents=True, add_demo=True)

In [6]:
# Check group composition in terms of male/female
n_male = len(male_group)
n_female = len(female_group)

print('Number of males in the group: \t\t', n_male)
print('Number of females in the group: \t', n_female)

Number of males in the group: 		 31
Number of females in the group: 	 30


In [7]:
# Check average ages between groups
print('Male: \t', np.mean(male_group.age))
print('Female: ', np.mean(female_group.age))

# Check if there is a significant difference in age between M & F
print('\n',ttest_ind(male_group.age, female_group.age))

Male: 	 29.451612903225808
Female:  29.633333333333333

 Ttest_indResult(statistic=-0.24071233340627646, pvalue=0.810612579474068)


In [8]:
# Calculate osc maps
female_group.osc_prob()
female_group.osc_power()
female_group.osc_score()
male_group.osc_prob()
male_group.osc_power()
male_group.osc_score()

In [9]:
# Difference group between sexes
diff_group = MegGroup(db, osc)

for band in osc.bands:
    diff_group.osc_probs[band] = female_group.osc_probs[band] - male_group.osc_probs[band]
    diff_group.osc_prob_done = True
    diff_group.osc_scores[band] = female_group.osc_scores[band] - male_group.osc_scores[band]
    diff_group.osc_score_done = True

In [10]:
# Check descriptive states of differences across space
for band in list(osc.bands):
    print(band)
    print('\tMax  diff: ', np.max(diff_group.osc_scores[band]))
    print('\tMin  diff: ', np.min(diff_group.osc_scores[band]))
    print('\tMean diff: ', np.mean(diff_group.osc_scores[band]))
    print('\tStat comp: ', ttest_1samp(diff_group.osc_scores[band], 0))
    
    
    #plt.figure()
    #plt.hist(diff_group.osc_scores[band])
    #plt.title(band)

Theta
	Max  diff:  0.11479786401578077
	Min  diff:  -0.3767940654080574
	Mean diff:  -0.03707996401315807
	Stat comp:  Ttest_1sampResult(statistic=-68.9569934879144, pvalue=0.0)
Alpha
	Max  diff:  0.42774502564328154
	Min  diff:  -0.11530313142450688
	Mean diff:  0.11029633645593226
	Stat comp:  Ttest_1sampResult(statistic=135.61195031361217, pvalue=0.0)
Beta
	Max  diff:  0.2812226038122778
	Min  diff:  -0.17340844112590004
	Mean diff:  0.056036992068983146
	Stat comp:  Ttest_1sampResult(statistic=62.178321127571586, pvalue=0.0)


In [11]:
# Set osc maps for viz
#diff_group.set_map_viz('score', 'diff')

In [12]:
# Exponents
avg = 'mean'
male_group.group_exponent(avg)
female_group.group_exponent(avg)

In [13]:
m_sls = np.mean(male_group.vert_exponents, 0)
f_sls = np.mean(female_group.vert_exponents, 0)

In [14]:
ttest_ind(m_sls, f_sls)

Ttest_indResult(statistic=1.2004674448486385, pvalue=0.23475566680754073)

In [15]:
avg_m_sl = np.mean(m_sls)
avg_fm_sl = np.mean(f_sls)
print(avg_m_sl)
print(avg_fm_sl)

0.8356120543058997
0.7916258792025126


In [16]:
np.mean(np.concatenate([m_sls, f_sls]))

0.813979509173086

In [17]:
diff_group.exponent_gr_avg = female_group.exponent_gr_avg - male_group.exponent_gr_avg

In [18]:
np.mean(diff_group.exponent_gr_avg)

-0.043986175103387606

In [19]:
ttest_1samp(diff_group.exponent_gr_avg, 0)

Ttest_1sampResult(statistic=-87.97148919945954, pvalue=0.0)